## SQL Class Testing

In [2]:
from sql import SQL
import pandas as pd
import re

### Data cleaning 

In [3]:
def extract_number(s):
    match = re.search(r"\d+\.?\d*", s) 
    return round(float(match.group()) / 100, 4)

# Clean locations.csv
df = pd.read_csv('data/locations.csv')[['name', 'longitude', 'latitude']].iloc[2:].reset_index(drop=True)
df.to_csv('data/locations.csv', index=False)

# Clean countries.csv
try:
    df = pd.read_csv('data/countries.csv').rename(columns={'Country or Area':'country', 
                                                        'Internet Users':'users', 
                                                        'Population':'population', 
                                                        'Rank':'rank',
                                                        'Percentage':'percentage'})[['country', 'users', 'population', 'rank', 'percentage']]
    df['users'] = df['users'].str.replace(',', '').astype(int)
    df['population'] = df['population'].str.replace(',', '').astype(int)
    df['percentage'] = df['percentage'].apply(extract_number)
    df.to_csv('data/countries.csv', index=False)
except:
    df = pd.read_csv('data/countries.csv')
    df['percentage'] = df['percentage'].round(4)
    df.to_csv('data/countries.csv', index=False)

### Testing

In [4]:
# add_table
database = SQL()
database.add_table('data/countries.csv', 'countries', ',')
database.add_table('data/locations.csv', 'locations', ',')

In [5]:
# Test: sql_from
table1, table1_name, mod_name1 = database.sql_from('FROM countries')
table2, table2_name, mod_name2 = database.sql_from('FROM locations')

for col in table1.keys():
    print(col, table1[col][:3])
print(mod_name1)

print()

for col in table2.keys():
    print(col, table2[col][:3])
print(mod_name2)

country ['China', 'India', 'United States']
users ['765367947', '461347554', '244090854']
population ['1409517397', '1339180127', '324459463']
rank ['1', '2', '3']
percentage ['0.543', '0.3445', '0.7523']
countries

name ['Hong Kong', 'Hungary', 'Iceland']
longitude ['22.396428', '47.162494', '64.963051']
latitude ['114.109497', '19.5033041', '-19.020835']
locations


In [6]:
# Test: sql_join

table3 = database.sql_join('JOIN locations ON countries.country = locations.name', table1_name, mod_name1)
table4 = database.sql_join('JOIN locations l ON c.country = l.name', table1_name, 'c')
table5 = database.sql_join('JOIN locations AS l ON countries.country = l.name', table1_name, mod_name1)

for col in table3.keys():
    print(col, table3[col][:3])

print()

for col in table4.keys():
    print(col, table4[col][:3])

print()

for col in table5.keys():
    print(col, table5[col][:3])

countries.country ['India', 'Japan', 'Russia']
countries.users ['461347554', '115845120', '109446612']
countries.population ['1339180127', '127484450', '143989754']
countries.rank ['2', '5', '6']
countries.percentage ['0.3445', '0.9087', '0.7601']
locations.name ['India', 'Japan', 'Russia']
locations.longitude ['20.593684', '36.204824', '61.52401']
locations.latitude ['78.96288', '138.252924', '105.318756']

c.country ['India', 'Japan', 'Russia']
c.users ['461347554', '115845120', '109446612']
c.population ['1339180127', '127484450', '143989754']
c.rank ['2', '5', '6']
c.percentage ['0.3445', '0.9087', '0.7601']
l.name ['India', 'Japan', 'Russia']
l.longitude ['20.593684', '36.204824', '61.52401']
l.latitude ['78.96288', '138.252924', '105.318756']

countries.country ['India', 'Japan', 'Russia']
countries.users ['461347554', '115845120', '109446612']
countries.population ['1339180127', '127484450', '143989754']
countries.rank ['2', '5', '6']
countries.percentage ['0.3445', '0.9087', '0

In [11]:
database.sql_select('SELECT c.country AS c, locations.name n', table1)
# database.sql_select('SELECT *', table1)

['c.country AS c', 'locations.name n']
